# Object Recognition Training Notebook (Caltech-101)

This notebook handles the downloading, processing, and training of a custom object detection model using the Caltech-101 dataset.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import os
import pathlib

## 1. Download and Prepare Data

In [ ]:
dataset_url = "https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip?download=1"
data_dir = tf.keras.utils.get_file('caltech-101', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).parent / "caltech-101" / "101_ObjectCategories"
print(f"Data loaded at: {data_dir}")

In [ ]:
# Clean up BACKGROUND_Google class if it exists (it's usually noise)
import shutil
background_dir = data_dir / "BACKGROUND_Google"
if background_dir.exists():
    shutil.rmtree(background_dir)
    print("Removed BACKGROUND_Google class")

## 2. Create Data Pipeline

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_ds.class_names
print(f"Number of classes: {len(class_names)}")

In [ ]:
# Pre-fetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

## 3. Build Model (Transfer Learning)

In [ ]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

## 4. Save Model and Labels

In [ ]:
model_dir = pathlib.Path("model")
model_dir.mkdir(exist_ok=True)

model.save(model_dir / "caltech101_model.h5")
print("Model saved to model/caltech101_model.h5")

with open(model_dir / "caltech101_labels.json", 'w') as f:
    json.dump(class_names, f)
print("Labels saved to model/caltech101_labels.json")